# Data Cleaning

In [1]:
import pandas as pd
import pycountry
import os

In [2]:
print(os.getcwd())

/Users/timoklein/Library/CloudStorage/OneDrive-Personal/Universiteit/Studie/M_Data_Science_and_Society/year_2/Thesis/Code/timoklein


## Clean the Technology Readiness Index dataset

In [3]:
## Load the Technology Readiness Index dataset into a DataFrame
df_tri = pd.read_csv('Data/Technology_Readiness_Index.csv')

# Only keep the columns 'Economy_Label', 'Category_Label', '2015_Index_Value' and '2016_Index_Value'
df_tri = df_tri[['Economy_Label', 'Category_Label', '2015_Index_Value', '2016_Index_Value']]

# Add a column with the average index value for 2015 and 2016
df_tri['Average_Index_Value'] = df_tri[['2015_Index_Value', '2016_Index_Value']].mean(axis=1)

# Delete the 2015 and 2016 index value columns
df_tri.drop(columns=['2015_Index_Value', '2016_Index_Value'], inplace=True)

# Pivot the dataframe
df_tri_cleaned = df_tri.pivot(index='Economy_Label', columns='Category_Label', values=['Average_Index_Value'])

# Flatten the multi-level column index
df_tri_cleaned.columns = ['_'.join(col).strip() for col in df_tri_cleaned.columns.values]

# Delete the row with 'Economy_Label'
df_tri_cleaned.reset_index(inplace=True)

# Rename the columns
df_tri_cleaned.columns = ['Country', 'Finance_access', 'ICT', 'Industry_activity', 'Overall_index', 'Research_and_development', 'Skills']

# Drop missing values
df_tri_cleaned.dropna(inplace=True)

# Change the 'Country' column to ISO codes
# Extract the 'Country' column as a list
country_names = df_tri_cleaned['Country'].tolist()

# Create a mapping of country names to ISO codes
country_to_iso = {}
for country in country_names:
    try:
        # Get the country object using the name
        country_obj = pycountry.countries.lookup(country)
        # Map country name to ISO code
        country_to_iso[country] = country_obj.alpha_3  # Using 3-letter ISO code
    except LookupError:
        # If the country is not found, handle it as needed
        print(f"Country not found: {country}")

# Manual mapping for countries that did not get an ISO code
manual_iso_mapping = {
    "Bolivia (Plurinational State of)": "BOL",
    "China, Hong Kong SAR": "HKG",
    "Congo, Dem. Rep. of the": "COD",
    "Cote d'Ivoire": "CIV",
    "Iran (Islamic Republic of)": "IRN",
    "Lao People's Dem. Rep.": "LAO",
    "Netherlands (Kingdom of the)": "NLD",
    "Switzerland, Liechtenstein": "CHE",  
    "Turkiye": "TUR",  
    "Venezuela (Bolivarian Rep. of)": "VEN"
}

# Update the mapping with the manual mappings
country_to_iso.update(manual_iso_mapping)

# Create a Series from the mapping to use for replacing values
iso_codes_series = pd.Series(country_to_iso)

# Replace the 'Country' column values with ISO codes
df_tri_cleaned['Country'] = df_tri_cleaned['Country'].replace(iso_codes_series)

df_tri_cleaned.head()

Country not found: Bolivia (Plurinational State of)
Country not found: China, Hong Kong SAR
Country not found: Congo, Dem. Rep. of the
Country not found: Cote d'Ivoire
Country not found: Iran (Islamic Republic of)
Country not found: Lao People's Dem. Rep.
Country not found: Netherlands (Kingdom of the)
Country not found: Switzerland, Liechtenstein
Country not found: Turkiye
Country not found: Venezuela (Bolivarian Rep. of)


,Country,Finance_access,ICT,Industry_activity,Overall_index,Research_and_development,Skills
0,AFG,0.15,0.00,0.25,0.00,0.00,0.10
1,ALB,0.60,0.50,0.50,0.40,0.10,0.45
2,DZA,0.50,0.25,0.20,0.30,0.30,0.40
3,AGO,0.50,0.10,0.25,0.15,0.05,0.20
4,ARG,0.40,0.55,0.60,0.55,0.30,0.65


In [4]:
df_tri_cleaned.describe()

,Finance_access,ICT,Industry_activity,Overall_index,Research_and_development,Skills
count,166.000000,166.000000,166.000000,166.000000,166.000000,166.000000
mean,0.628614,0.423494,0.574699,0.450904,0.246084,0.432229
std,0.180766,0.249768,0.193199,0.262648,0.240580,0.238875
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.500000,0.200000,0.400000,0.200000,0.050000,0.212500
50%,0.650000,0.450000,0.600000,0.400000,0.150000,0.400000
75%,0.750000,0.600000,0.700000,0.650000,0.400000,0.600000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


## Clean the Road Traffic Deaths Sex dataset

In [5]:
## Load the Road Traffic Deaths Sex dataset into a DataFrame
df_rtds = pd.read_csv('Data/Road_Traffic_Deaths_Sex.csv')

# Only keep the columns 'Unnamed: 0', 'Unnamed: 1', 'Estimated road traffic death rate (per 100 000 population)', 'Estimated road traffic death rate (per 100 000 population).1' and 'Estimated road traffic death rate (per 100 000 population).2'
df_rtds = df_rtds[['Unnamed: 0', 'Unnamed: 1', 'Estimated road traffic death rate (per 100 000 population)', 'Estimated road traffic death rate (per 100 000 population).1', 'Estimated road traffic death rate (per 100 000 population).2']]

# Rename the columns and delete the first row
df_rtds.columns = ['Country', 'Year', 'Total', 'Males', 'Females']
df_rtds = df_rtds.iloc[1:]

# Only keep the rows where the 'Year' column is '2015' or '2016'
df_rtds['Year'] = df_rtds['Year'].str.strip()
df_rtds = df_rtds[df_rtds['Year'].isin(['2015', '2016'])]

# Remove the numbers between square brackets
columns_to_clean = ['Total','Males','Females']

for col in columns_to_clean:
    df_rtds[col] = df_rtds[col].str.replace(r'\s*\[.*?\]', '', regex=True)

# Convert the death rate columns to numeric
df_rtds['Total'] = pd.to_numeric(df_rtds['Total'], errors='coerce')
df_rtds['Males'] = pd.to_numeric(df_rtds['Males'], errors='coerce')
df_rtds['Females'] = pd.to_numeric(df_rtds['Females'], errors='coerce')
df_rtds['Year'] = pd.to_numeric(df_rtds['Year'], errors='coerce')

# Group by 'Country' and calculate the mean for the last three columns for '2015' and '2016'
df_rtds = df_rtds.groupby('Country').mean()[['Total','Males','Females']]

# Normalize the last three columns 
# Convert columns to numeric type
columns_to_normalize = ['Total','Males','Females']

for col in columns_to_normalize:
    df_rtds[col] = pd.to_numeric(df_rtds[col])

# Min-Max Normalization directly in the existing DataFrame
#for col in columns_to_normalize:
#    df_rtds[col] = (df_rtds[col] - df_rtds[col].min()) / (df_rtds[col].max() - df_rtds[col].min())

# Reset the index so that "Country" becomes a regular column
df_rtds = df_rtds.reset_index()

# Change the 'Country' column to ISO codes
# Extract the 'Country' column as a list
country_names = df_rtds['Country'].tolist()

# Create a mapping of country names to ISO codes
country_to_iso = {}
for country in country_names:
    try:
        # Get the country object using the name
        country_obj = pycountry.countries.lookup(country)
        # Map country name to ISO code
        country_to_iso[country] = country_obj.alpha_3  # Using 3-letter ISO code
    except LookupError:
        # If the country is not found, handle it as needed
        print(f"Country not found: {country}")

# Manual mapping for countries that did not get an ISO code
manual_iso_mapping = {
    "Bolivia (Plurinational State of)": "BOL",
    "Cote d'Ivoire": "CIV",  
    "Democratic Republic of the Congo": "COD",
    "Iran (Islamic Republic of)": "IRN",
    "Micronesia (Federated States of)": "FSM",
    "Netherlands (Kingdom of the)": "NLD",
    "Republic of Korea": "KOR",
    "Turkiye": "TUR",  
    "Venezuela (Bolivarian Republic of)": "VEN"
}

# Update the mapping with the manual mappings
country_to_iso.update(manual_iso_mapping)

# Create a Series from the mapping to use for replacing values
iso_codes_series = pd.Series(country_to_iso)

# Replace the 'Country' column values with ISO codes
df_rtds['Country'] = df_rtds['Country'].replace(iso_codes_series)

df_rtds.head(10)


Country not found: Bolivia (Plurinational State of)
Country not found: Cote d'Ivoire
Country not found: Democratic Republic of the Congo
Country not found: Iran (Islamic Republic of)
Country not found: Micronesia (Federated States of)
Country not found: Netherlands (Kingdom of the)
Country not found: Republic of Korea
Country not found: Turkiye
Country not found: Venezuela (Bolivarian Republic of)


,Country,Total,Males,Females
0,AFG,14.60,22.90,5.75
1,ALB,13.80,20.90,6.45
2,DZA,21.15,28.40,13.70
3,AGO,24.15,30.80,17.75
4,ATG,0.00,0.00,0.00
5,ARG,14.00,22.35,6.00
6,ARM,16.20,25.10,8.35
7,AUS,5.40,7.80,2.95
8,AUT,5.45,7.95,3.00
9,AZE,9.50,15.25,3.80


In [6]:
df_rtds.describe()

,Total,Males,Females
count,183.000000,183.000000,183.000000
mean,17.063934,25.370765,8.760383
std,9.388973,13.848550,5.802119
min,0.000000,0.000000,0.000000
25%,9.500000,14.450000,4.175000
50%,16.050000,24.250000,7.250000
75%,24.775000,35.500000,13.025000
max,39.300000,64.400000,23.900000


## Clean the Road Traffic Deaths User dataset

In [7]:
## Read the Road Traffic Deaths Sex dataset into a DataFrame
df_rtdsfu = pd.read_csv('Data/Road_Traffic_Deaths_Sex.csv')

## Read the Road Traffic Deaths User dataset into a DataFrame
df_rtdu = pd.read_csv('Data/Road_Traffic_Deaths_User.csv')

# Only keep the columns 'Unnamed: 0' and 'Estimated number of road traffic deaths'
df_rtdsfu = df_rtdsfu[['Unnamed: 0','Unnamed: 1','Estimated road traffic death rate (per 100 000 population)']]

# Rename the columns and delete the first row
df_rtdsfu.columns = ['Country', 'Year','Total']
df_rtdsfu = df_rtdsfu.iloc[1:]

# Only keep the rows where the 'Year' column is '2016'
df_rtdsfu['Year'] = df_rtdsfu['Year'].str.strip()
df_rtdsfu = df_rtdsfu[df_rtdsfu['Year'].isin(['2016'])]

# Remove the numbers between square brackets
df_rtdsfu['Total'] = df_rtdsfu['Total'].str.replace(r'\s*\[.*?\]', '', regex=True)

# Delete the 'Year' column
df_rtdsfu.drop(columns=['Year'], inplace=True)

# Only keep the columns 'Unnamed: 0','Unnamed: 1', 'Distribution of road traffic deaths by type of road user (%)' and 'Distribution of road traffic deaths by type of road user (%).3'
df_rtdu = df_rtdu[['Unnamed: 0','Unnamed: 1','Distribution of road traffic deaths by type of road user (%)', 'Distribution of road traffic deaths by type of road user (%).3']]

# Rename the columns and delete the first row
df_rtdu.columns = ['Country', 'Year','Passengers', 'Pedestrians']
df_rtdu = df_rtdu.iloc[1:]

# Only keep the rows where the 'Year' column is '2016'
df_rtdu['Year'] = df_rtdu['Year'].str.strip()
df_rtdu = df_rtdu[df_rtdu['Year'].isin(['2016'])]

# Remove the 'h' from the 'Passengers' column
df_rtdu['Passengers'] = df_rtdu['Passengers'].str.replace('h', '', regex=False)

# Delete missing values
df_rtdu.dropna(inplace=True)

# Add the column 'Estimated_number_of_road_traffic_deaths' from the 'df_rtdsfu' DataFrame to the 'df_rtdu' DataFrame
df_rtdu_cleaned = pd.merge(df_rtdu, df_rtdsfu, on='Country', how='inner')

# Convert the 'Passengers', 'Pedestrians' and 'Total' columns to numeric
df_rtdu_cleaned['Passengers'] = pd.to_numeric(df_rtdu_cleaned['Passengers'], errors='coerce')
df_rtdu_cleaned['Pedestrians'] = pd.to_numeric(df_rtdu_cleaned['Pedestrians'], errors='coerce')
df_rtdu_cleaned['Total'] = pd.to_numeric(df_rtdu_cleaned['Total'], errors='coerce')

# Add a column with the estimated number of road traffic deaths for passengers and pedestrians
df_rtdu_cleaned['Passengers'] = df_rtdu_cleaned['Passengers']/100 * df_rtdu_cleaned['Total']
df_rtdu_cleaned['Pedestrians'] = df_rtdu_cleaned['Pedestrians']/100 * df_rtdu_cleaned['Total']

# Delete the 'Year' and 'Total' column
df_rtdu_cleaned.drop(columns=['Year','Total'], inplace=True)

# Convert columns to numeric type
columns_to_normalize = ['Passengers','Pedestrians']

for col in columns_to_normalize:
    df_rtdu_cleaned[col] = pd.to_numeric(df_rtdu_cleaned[col])

# Min-Max Normalization directly in the existing DataFrame
#for col in columns_to_normalize:
#    df_rtdu_cleaned[col] = (df_rtdu_cleaned[col] - df_rtdu_cleaned[col].min()) / (df_rtdu_cleaned[col].max() - df_rtdu_cleaned[col].min())

# Change the 'Country' column to ISO codes
# Extract the 'Country' column as a list
country_names = df_rtdu_cleaned['Country'].tolist()

# Create a mapping of country names to ISO codes
country_to_iso = {}
for country in country_names:
    try:
        # Get the country object using the name
        country_obj = pycountry.countries.lookup(country)
        # Map country name to ISO code
        country_to_iso[country] = country_obj.alpha_3  # Using 3-letter ISO code
    except LookupError:
        # If the country is not found, handle it as needed
        print(f"Country not found: {country}")

# Manual mapping for countries that did not get an ISO code
manual_iso_mapping = {
    "Bolivia (Plurinational State of)": "BOL",
    "Cote d'Ivoire": "CIV",  # Côte d'Ivoire
    "Democratic Republic of the Congo": "COD",
    "Iran (Islamic Republic of)": "IRN",
    "Micronesia (Federated States of)": "FSM",
    "Netherlands (Kingdom of the)": "NLD",
    "Republic of Korea": "KOR"  # Use KOR for South Korea
}

# Update the mapping with the manual mappings
country_to_iso.update(manual_iso_mapping)

# Create a Series from the mapping to use for replacing values
iso_codes_series = pd.Series(country_to_iso)

# Replace the 'Country' column values with ISO codes
df_rtdu_cleaned['Country'] = df_rtdu_cleaned['Country'].replace(iso_codes_series)

df_rtdu_cleaned.tail(10)

Country not found: Bolivia (Plurinational State of)
Country not found: Cote d'Ivoire
Country not found: Democratic Republic of the Congo
Country not found: Iran (Islamic Republic of)
Country not found: Micronesia (Federated States of)
Country not found: Netherlands (Kingdom of the)
Country not found: Republic of Korea


,Country,Passengers,Pedestrians
107,THA,15.1450,2.4700
108,TON,10.2884,4.9484
109,TTO,5.9160,3.7320
110,TUN,4.9910,5.6580
111,UGA,10.3360,12.0080
112,UKR,7.4120,5.6984
113,ARE,8.2698,4.3497
114,GBR,1.1550,0.7110
115,TZA,19.5534,9.1494
116,USA,3.9603,1.9737


In [8]:
df_rtdu_cleaned.describe()

,Passengers,Pedestrians
count,117.000000,117.000000
mean,5.613485,4.193465
std,4.716367,3.757972
min,0.000000,0.000000
25%,2.277600,1.435000
50%,4.167600,3.633300
75%,7.126200,5.567100
max,19.553400,17.792500


## Clean the Moral Machine dataset

In [9]:
# Load the 'ExtendedSessionID' column from the Moral Machine dataset into a DataFrame
extendedsessionid = pd.read_csv('Data/SharedResponses.csv', usecols=['ExtendedSessionID'])

In [10]:
# Filter out the sessions that have between 24 and 26 responses
session_counts = extendedsessionid['ExtendedSessionID'].value_counts()
ids_to_keep = session_counts[(session_counts >= 24) & (session_counts <= 26)].index

print(f"Number of session ids to keep: {len(ids_to_keep)}")

Number of session ids to keep: 1105760


In [11]:
df_mm = pd.read_csv('Data/SharedResponses.csv', chunksize=100_000, dtype=str, low_memory=False)

cleaned_chunks = []
for i, chunk in enumerate(df_mm):
    print(f"Processing chunk {i+1}")
    
    # Cleaning 
    chunk_cleaned = chunk[chunk['ExtendedSessionID'].isin(ids_to_keep)]
    chunk_cleaned = chunk_cleaned.drop(columns=['ScenarioOrder', 'Intervention', 'ScenarioType', 'DefaultChoice', 'NonDefaultChoice', 'DefaultChoiceIsOmission', 'Template','DescriptionShown','LeftHand'])
    chunk_cleaned.rename(columns={'UserCountry3': 'Country'}, inplace=True)
    chunk_cleaned = chunk_cleaned.dropna()
    chunk_cleaned = chunk_cleaned.drop_duplicates()
    chunk_cleaned = chunk_cleaned.merge(df_tri_cleaned, on='Country', how='inner').merge(df_rtds, on='Country', how='inner').merge(df_rtdu_cleaned, on='Country', how='inner')
    chunk_cleaned[['ResponseID', 'ExtendedSessionID', 'UserID', 'AttributeLevel','ScenarioTypeStrict']] = chunk_cleaned[['ResponseID', 'ExtendedSessionID', 'UserID', 'AttributeLevel','ScenarioTypeStrict']].astype(str)
    chunk_cleaned[["PedPed", "Barrier", "CrossingSignal", "NumberOfCharacters", "DiffNumberOFCharacters", "Man", "Woman", "Pregnant", "Stroller", "OldMan", "OldWoman", "Boy", "Girl", "Homeless", "LargeWoman", "LargeMan", "Criminal", "MaleExecutive", "FemaleExecutive", "FemaleAthlete", "MaleAthlete", "FemaleDoctor", "MaleDoctor", "Dog", "Cat", "Saved"]] = chunk_cleaned[["PedPed", "Barrier", "CrossingSignal", "NumberOfCharacters", "DiffNumberOFCharacters", "Man", "Woman", "Pregnant", "Stroller", "OldMan", "OldWoman", "Boy", "Girl", "Homeless", "LargeWoman", "LargeMan", "Criminal", "MaleExecutive", "FemaleExecutive", "FemaleAthlete", "MaleAthlete", "FemaleDoctor", "MaleDoctor", "Dog", "Cat", "Saved"]].astype(float).round().astype('int8')
    chunk_cleaned[["Finance_access", "ICT", "Industry_activity", "Overall_index", "Research_and_development", "Skills", "Total", "Males", "Females", "Passengers", "Pedestrians"]] = chunk_cleaned[["Finance_access", "ICT", "Industry_activity", "Overall_index", "Research_and_development", "Skills", "Total", "Males", "Females", "Passengers", "Pedestrians"]].astype('float32')

    # Append cleaned chunk
    cleaned_chunks.append(chunk_cleaned)
    print(f"Finished processing chunk {i+1}")

    # Stop after processing five chunks
    #j = 5
    #if i + 1 == j:
    #    print(f"Stopping after processing {j} chunks")
    #    break

print("All chunks have been processed")

Processing chunk 1
Finished processing chunk 1
Processing chunk 2
Finished processing chunk 2
Processing chunk 3
Finished processing chunk 3
Processing chunk 4
Finished processing chunk 4
Processing chunk 5
Finished processing chunk 5
Processing chunk 6
Finished processing chunk 6
Processing chunk 7
Finished processing chunk 7
Processing chunk 8
Finished processing chunk 8
Processing chunk 9
Finished processing chunk 9
Processing chunk 10
Finished processing chunk 10
Processing chunk 11
Finished processing chunk 11
Processing chunk 12
Finished processing chunk 12
Processing chunk 13
Finished processing chunk 13
Processing chunk 14
Finished processing chunk 14
Processing chunk 15
Finished processing chunk 15
Processing chunk 16
Finished processing chunk 16
Processing chunk 17
Finished processing chunk 17
Processing chunk 18
Finished processing chunk 18
Processing chunk 19
Finished processing chunk 19
Processing chunk 20
Finished processing chunk 20
Processing chunk 21
Finished processin

In [12]:
cleaned_data = pd.concat(cleaned_chunks, ignore_index=True)

In [13]:
responseid_counts = cleaned_data['ResponseID'].value_counts()
responseid_to_keep = responseid_counts[responseid_counts == 2].index

In [14]:
cleaned_data = cleaned_data[cleaned_data['ResponseID'].isin(responseid_to_keep)]

In [15]:
# Get the value counts for each ExtendedSessionID
session_counts = cleaned_data['ExtendedSessionID'].value_counts()

# Step 1: Calculate the counts for 26, 25, and 24 occurrences
count_26 = (session_counts == 26).sum()
count_25 = (session_counts == 25).sum()
count_24 = (session_counts == 24).sum()

# Step 2: Set the target number of rows (10 million)
target_rows = 10_000_000

# Step 3: Calculate the number of rows if we keep all ExtendedSessionIDs appearing 26 times
rows_from_26 = count_26 * 26

# Step 4: If keeping all ExtendedSessionIDs appearing 26 times keeps us under the target:
if rows_from_26 <= target_rows:
    # Keep all ExtendedSessionIDs appearing 26 times
    ids_to_keep = session_counts[session_counts == 26].index
    remaining_rows = target_rows - rows_from_26
    
    # Step 5: Calculate the number of rows if we also keep all ExtendedSessionIDs appearing 25 times
    rows_from_25 = count_25 * 25
    if rows_from_25 <= remaining_rows:
        # Keep all ExtendedSessionIDs appearing 25 times
        ids_to_keep = ids_to_keep.union(session_counts[session_counts == 25].index)
        remaining_rows -= rows_from_25
        
        # Step 6: Calculate the number of rows if we also keep all ExtendedSessionIDs appearing 24 times
        rows_from_24 = count_24 * 24
        if rows_from_24 <= remaining_rows:
            # Keep all ExtendedSessionIDs appearing 24 times
            ids_to_keep = ids_to_keep.union(session_counts[session_counts == 24].index)
        else:
            # Keep only enough ExtendedSessionIDs appearing 24 times to reach the target
            num_to_keep_24 = remaining_rows // 24
            ids_to_keep = ids_to_keep.union(session_counts[session_counts == 24].index[:num_to_keep_24])
    else:
        # Keep only enough ExtendedSessionIDs appearing 25 times to reach the target
        num_to_keep_25 = remaining_rows // 25
        ids_to_keep = ids_to_keep.union(session_counts[session_counts == 25].index[:num_to_keep_25])
else:
    # Keep only enough ExtendedSessionIDs appearing 26 times to reach the target
    num_to_keep_26 = target_rows // 26
    ids_to_keep = session_counts[session_counts == 26].index[:num_to_keep_26]

# Step 7: Filter the original dataset to keep only the selected ExtendedSessionIDs
filtered_sessions = extendedsessionid[extendedsessionid['ExtendedSessionID'].isin(ids_to_keep)]
filtered_sessions = (filtered_sessions['ExtendedSessionID'].unique()).tolist()
print(type(filtered_sessions))

# Display the filtered dataset
print(len(filtered_sessions))

<class 'list'>
412256


In [16]:
cleaned_data = cleaned_data[cleaned_data['ExtendedSessionID'].isin(filtered_sessions)]

## Looking at the final dataset

In [17]:
pd.set_option('display.max_columns', None)
cleaned_data.head(10)

,ResponseID,ExtendedSessionID,UserID,PedPed,Barrier,CrossingSignal,AttributeLevel,ScenarioTypeStrict,NumberOfCharacters,DiffNumberOFCharacters,Saved,Country,Man,Woman,Pregnant,Stroller,OldMan,OldWoman,Boy,Girl,Homeless,LargeWoman,LargeMan,Criminal,MaleExecutive,FemaleExecutive,FemaleAthlete,MaleAthlete,FemaleDoctor,MaleDoctor,Dog,Cat,Finance_access,ICT,Industry_activity,Overall_index,Research_and_development,Skills,Total,Males,Females,Passengers,Pedestrians
0,2223Xu54ufgjcyMR3,1425316635_327833569077076.0,327833569077076,0,1,0,Old,Age,5,0,0,MEX,0,0,0,0,2,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.60,0.55,0.80,0.60,0.5,0.40,13.60,22.049999,5.45,2.5024,3.8760
1,2223jMWDEGNeszivb,-1683127088_785070916172117.0,785070916172117,1,0,2,More,Utilitarian,5,2,0,CHE,0,0,0,0,1,0,0,0,0,0,0,1,1,0,1,0,0,0,1,0,0.90,0.65,0.90,0.90,0.7,0.80,2.90,4.200000,1.60,0.5076,0.6237
25,222HpiEf2LtAwEg62,-1232628507_1597557389,1597557389,0,1,0,Female,Gender,2,0,0,UKR,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0.75,0.50,0.65,0.65,0.5,0.65,13.25,21.750000,5.95,7.4120,5.6984
28,222KuWty7pNeiv77a,1654911454_3639764894860440.0,3639764894860440,1,0,0,Low,Social Status,2,0,0,USA,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0.90,0.65,0.80,1.00,1.0,0.75,12.50,17.850000,7.25,3.9603,1.9737
30,222LDp4wz24C3chzj,-1679158262_3623236506.0,3623236506,0,0,0,Fat,Fitness,2,0,0,DEU,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0.80,0.80,0.90,0.90,0.8,0.75,4.20,6.350000,2.15,1.9120,0.6120
51,222dZwp7jYt7FrkfQ,781757349_7305361930957958.0,7305361930957960,1,0,0,Male,Gender,1,0,0,USA,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.90,0.65,0.80,1.00,1.0,0.75,12.50,17.850000,7.25,3.9603,1.9737
55,222fkCAzoe6MAnMsP,-624226515_2260272466.0,2260272466,0,0,0,Female,Gender,1,0,1,NLD,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0.80,0.80,0.90,0.95,0.7,0.85,3.80,5.650000,1.95,1.4440,0.3496
70,222t9Qtkcc4EnQvcc,1829270983_2465676825.0,2465676825,1,0,0,More,Utilitarian,5,2,1,CAN,0,0,0,0,1,1,0,0,0,0,0,0,0,0,2,0,0,0,0,1,0.90,0.80,0.80,0.90,0.7,0.75,5.80,8.150000,3.45,3.7294,0.8816
90,223BYifT53tQSv7Yg,-1594341876_1841400032020538.0,1841400032020540,1,0,2,Young,Age,3,0,0,USA,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0.90,0.65,0.80,1.00,1.0,0.75,12.50,17.850000,7.25,3.9603,1.9737
105,223NQzNqcbab7XWQn,-409747588_5976174606267197.0,5976174606267200,1,0,1,More,Utilitarian,5,1,1,AZE,0,0,0,1,0,0,0,0,1,0,0,2,0,1,0,0,0,0,0,0,0.60,0.50,0.35,0.30,0.1,0.35,9.50,15.250000,3.80,4.5066,3.6540


In [19]:
extendedsessionid_values = cleaned_data['ExtendedSessionID'].value_counts()
extendedsessionid_values

ExtendedSessionID
-1515635037_7014932942427610.0    26
-649792789_123617179              26
-1631266300_7001055290384566.0    26
-551512359_4144432253.0           26
-451394117_519359821219711.0      26
                                  ..
1196203245_9395865181510246.0     24
-417777874_2014408738             24
-401464432_2285117361.0           24
1656669350_9046809254077914.0     24
551980216_3319982808.0            24
Name: count, Length: 412256, dtype: int64

In [20]:
# Number of value_count is 24, 25 and 26
count_26 = (extendedsessionid_values == 26).sum()
count_25 = (extendedsessionid_values == 25).sum()
count_24 = (extendedsessionid_values == 24).sum()
print(count_26)
print(count_25)
print(count_24)

52924
0
359332


In [21]:
count_once = (extendedsessionid_values < 24).sum()
count_once

0

In [22]:
most_frequent_value = cleaned_data['ResponseID'].value_counts().idxmax()
max_value_count = cleaned_data['ResponseID'].value_counts().max()
least_frequent_value = cleaned_data['ResponseID'].value_counts().idxmin()
min_value_count = cleaned_data['ResponseID'].value_counts().min()

print(f"Most frequent value: {most_frequent_value}, Count: {max_value_count}")

Most frequent value: 2223Xu54ufgjcyMR3, Count: 2


In [23]:
print(f"Least frequent value: {least_frequent_value}, Count: {min_value_count}")

Least frequent value: 2223Xu54ufgjcyMR3, Count: 2


In [24]:
cleaned_data.describe()

,PedPed,Barrier,CrossingSignal,NumberOfCharacters,DiffNumberOFCharacters,Saved,Man,Woman,Pregnant,Stroller,OldMan,OldWoman,Boy,Girl,Homeless,LargeWoman,LargeMan,Criminal,MaleExecutive,FemaleExecutive,FemaleAthlete,MaleAthlete,FemaleDoctor,MaleDoctor,Dog,Cat,Finance_access,ICT,Industry_activity,Overall_index,Research_and_development,Skills,Total,Males,Females,Passengers,Pedestrians
count,9.999992e+06,9.999992e+06,9.999992e+06,9.999992e+06,9.999992e+06,9999992.0,9.999992e+06,9.999992e+06,9.999992e+06,9.999992e+06,9.999992e+06,9.999992e+06,9.999992e+06,9.999992e+06,9.999992e+06,9.999992e+06,9.999992e+06,9.999992e+06,9.999992e+06,9.999992e+06,9.999992e+06,9.999992e+06,9.999992e+06,9.999992e+06,9.999992e+06,9.999992e+06,9.999992e+06,9.999992e+06,9.999992e+06,9.999992e+06,9.999992e+06,9.999992e+06,9.999992e+06,9.999992e+06,9.999992e+06,9.999992e+06,9.999992e+06
mean,4.517082e-01,2.741459e-01,6.061092e-01,2.990266e+00,5.458598e-01,0.5,3.102753e-01,3.106293e-01,5.972915e-02,5.703515e-02,1.850920e-01,1.852236e-01,1.541063e-01,1.541007e-01,1.050380e-01,1.541672e-01,1.540358e-01,5.683015e-02,1.052104e-01,1.053253e-01,1.849366e-01,1.847790e-01,9.488708e-02,9.502938e-02,1.667987e-01,1.670362e-01,8.163826e-01,6.750644e-01,7.947744e-01,8.634647e-01,7.364607e-01,7.264218e-01,9.046264e+00,1.368448e+01,4.554417e+00,3.267029e+00,1.656296e+00
std,4.976625e-01,4.460829e-01,8.147176e-01,1.482857e+00,1.129296e+00,0.5,5.908965e-01,5.913108e-01,2.556587e-01,2.502378e-01,5.156442e-01,5.156233e-01,4.404961e-01,4.402197e-01,3.794025e-01,4.403530e-01,4.400539e-01,2.497549e-01,3.452981e-01,3.452887e-01,5.154392e-01,5.151208e-01,3.291838e-01,3.295424e-01,5.566472e-01,5.579227e-01,1.036725e-01,1.253239e-01,8.401662e-02,1.371994e-01,2.141902e-01,1.230905e-01,5.086800e+00,8.133916e+00,2.369699e+00,1.574745e+00,1.096867e+00
min,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,5.000000e-02,2.500000e-01,0.000000e+00,0.000000e+00,0.000000e+00,1.750000e+00,2.300000e+00,9.000000e-01,0.000000e+00,1.220000e-01
25%,0.000000e+00,0.000000e+00,0.000000e+00,2.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,7.500000e-01,6.500000e-01,8.000000e-01,8.000000e-01,6.000000e-01,6.500000e-01,4.900000e+00,7.500000e+00,2.450000e+00,1.932000e+00,7.840000e-01
50%,0.000000e+00,0.000000e+00,0.000000e+00,3.000000e+00,0.000000e+00,0.5,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,8.500000e-01,6.500000e-01,8.000000e-01,9.000000e-01,7.500000e-01,7.500000e-01,7.800000e+00,1.240000e+01,3.550000e+00,3.574200e+00,1.435000e+00
75%,1.000000e+00,1.000000e+00,1.000000e+00,4.000000e+00,0.000000e+00,1.0,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,9.000000e-01,8.000000e-01,8.000000e-01,1.000000e+00,1.000000e+00,7.500000e-01,1.250000e+01,1.785000e+01,7.250000e+00,3.960300e+00,1.973700e+00
max,1.000000e+00,1.000000e+00,2.000000e+00,5.000000e+00,4.000000e+00,1.0,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.00000

In [18]:
cleaned_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9999992 entries, 0 to 24102222
Data columns (total 43 columns):
 #   Column                    Dtype  
---  ------                    -----  
 0   ResponseID                object 
 1   ExtendedSessionID         object 
 2   UserID                    object 
 3   PedPed                    int8   
 4   Barrier                   int8   
 5   CrossingSignal            int8   
 6   AttributeLevel            object 
 7   ScenarioTypeStrict        object 
 8   NumberOfCharacters        int8   
 9   DiffNumberOFCharacters    int8   
 10  Saved                     int8   
 11  Country                   object 
 12  Man                       int8   
 13  Woman                     int8   
 14  Pregnant                  int8   
 15  Stroller                  int8   
 16  OldMan                    int8   
 17  OldWoman                  int8   
 18  Boy                       int8   
 19  Girl                      int8   
 20  Homeless                  in

In [25]:
# Check for missing values
missing_values = cleaned_data.isnull().sum()
missing_values

ResponseID                  0
ExtendedSessionID           0
UserID                      0
PedPed                      0
Barrier                     0
CrossingSignal              0
AttributeLevel              0
ScenarioTypeStrict          0
NumberOfCharacters          0
DiffNumberOFCharacters      0
Saved                       0
Country                     0
Man                         0
Woman                       0
Pregnant                    0
Stroller                    0
OldMan                      0
OldWoman                    0
Boy                         0
Girl                        0
Homeless                    0
LargeWoman                  0
LargeMan                    0
Criminal                    0
MaleExecutive               0
FemaleExecutive             0
FemaleAthlete               0
MaleAthlete                 0
FemaleDoctor                0
MaleDoctor                  0
Dog                         0
Cat                         0
Finance_access              0
ICT       

In [26]:
cleaned_data.columns

Index(['ResponseID', 'ExtendedSessionID', 'UserID', 'PedPed', 'Barrier',
       'CrossingSignal', 'AttributeLevel', 'ScenarioTypeStrict',
       'NumberOfCharacters', 'DiffNumberOFCharacters', 'Saved', 'Country',
       'Man', 'Woman', 'Pregnant', 'Stroller', 'OldMan', 'OldWoman', 'Boy',
       'Girl', 'Homeless', 'LargeWoman', 'LargeMan', 'Criminal',
       'MaleExecutive', 'FemaleExecutive', 'FemaleAthlete', 'MaleAthlete',
       'FemaleDoctor', 'MaleDoctor', 'Dog', 'Cat', 'Finance_access', 'ICT',
       'Industry_activity', 'Overall_index', 'Research_and_development',
       'Skills', 'Total', 'Males', 'Females', 'Passengers', 'Pedestrians'],
      dtype='object')

In [27]:
AttributeLevel_distribution = cleaned_data['AttributeLevel'].value_counts(normalize=True) * 100
ScenarioTypeStrict_distribution = cleaned_data['ScenarioTypeStrict'].value_counts(normalize=True) * 100
NumberOfCharacters_distribution = cleaned_data['NumberOfCharacters'].value_counts(normalize=True) * 100
DiffNumberOFCharacters_distribution = cleaned_data['DiffNumberOFCharacters'].value_counts(normalize=True) * 100

print(AttributeLevel_distribution)
print(ScenarioTypeStrict_distribution)
print(NumberOfCharacters_distribution)
print(DiffNumberOFCharacters_distribution)

AttributeLevel
Rand       11.817409
More        8.499467
Less        8.499467
Pets        8.279137
Hoomans     8.279137
Female      8.252817
Male        8.252817
Old         7.977286
Young       7.977286
Fat         7.444226
Fit         7.444226
Low         3.638363
High        3.638363
Name: proportion, dtype: float64
ScenarioTypeStrict
Utilitarian      16.503373
Gender           16.493593
Species          16.493513
Fitness          16.493493
Age              16.492013
Social Status     9.280227
Random            8.243787
Name: proportion, dtype: float64
NumberOfCharacters
5    23.192669
1    22.695198
2    19.156355
3    17.767794
4    17.187984
Name: proportion, dtype: float64
DiffNumberOFCharacters
0    76.877342
1     6.752025
2     6.092565
3     5.463444
4     4.814624
Name: proportion, dtype: float64


In [28]:
cleaned_data.to_csv('Data/cleaned_data.csv', index=False)